In [1]:
#imports
import os
import sys
import numpy as np
import scipy.io
import imageio
import scipy.misc
import tensorflow as tf
import random
#tensorflow slim is great tensor-api for use easely!
import tensorflow.contrib.slim as slim

#color data set load for learm
#It’s okay? I dont know but I use it fucker!
from learn_color_balance import load_ground_truth





/home/ees/anaconda3/envs/tensorflow_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
OUTPUT_DIR = "out/"

GT_DIR = "./gt/"
ORI_DIR ="./nothing/"



In [3]:
from PIL import Image
im = Image.open("./nothing/0001_nothing.jpg")
print(im.mode)

RGB


In [4]:

def load_vgg_model(param_path):
    data = scipy.io.loadmat(param_path)

    # read meta info
    meta = data['meta']
    classes = meta['classes']
    class_names = classes[0][0]['description'][0][0]
    normalization = meta['normalization']
    average_image = np.squeeze(normalization[0][0]['averageImage'][0][0][0][0])
    image_size = np.squeeze(normalization[0][0]['imageSize'][0][0])
    #input_maps = tf.image.resize_images(input_maps, image_size[0], image_size[1])
    
    # read layer info
    layers = data['layers']
    network = {}
    network['input']   = tf.Variable(np.zeros((1, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS)), dtype = 'float32')
    current = network['input']
    for layer in layers[0]:
        name = layer[0]['name'][0][0]
        layer_type = layer[0]['type'][0][0]
        if layer_type == 'conv':
            if name[:2] == 'fc':
                padding = 'VALID'
            else:
                padding = 'SAME'
            stride = layer[0]['stride'][0][0]
            kernel, bias = layer[0]['weights'][0][0]
            # kernel = np.transpose(kernel, (1, 0, 2, 3))
            bias = np.squeeze(bias).reshape(-1)
            conv = tf.nn.conv2d(current, tf.constant(kernel),
                                strides=(1, stride[0], stride[0], 1), padding=padding)
            current = tf.nn.bias_add(conv, bias)
            #print name, 'stride:', stride, 'kernel size:', np.shape(kernel)
        elif layer_type == 'relu':
            current = tf.nn.relu(current)
            #print name
        elif layer_type == 'pool':
            stride = layer[0]['stride'][0][0]
            pool = layer[0]['pool'][0][0]
            current = tf.nn.max_pool(current, ksize=(1, pool[0], pool[1], 1),
                                     strides=(1, stride[0], stride[0], 1), padding='SAME')
            #print name, 'stride:', stride
        elif layer_type == 'softmax':
            current = tf.nn.softmax(tf.reshape(current, [-1, len(class_names)]))
            #print name

        print("layer name : "+name)
        network[name] = current
    
    #result is 3 label! gamma 0, 1, 2 What is it? IDONTKNOW DO NOT ASK ME
    W_fc1 = current
    current=tf.matmul(current, [])

    return network

In [5]:
def load_gt(param):
    (gt_illuminants,black_levels) = load_ground_truth(param)
    return gt_illuminants
#gt_il... is 1x3 table

In [6]:
il = load_gt("real_illum_568..mat")


In [7]:
il[0]

array([ 769., 1043.,  653.])

In [8]:
def load_inception_model(frozen_graph_filename, width, height):
    # We load the protobuf file from the disk and parse it to retrieve the
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        #pl = tf.placeholder(tf.float32, [None, width, height, 3], name= "input")
        tf.import_graph_def(
            graph_def,
            #input_map={"input": pl},
            return_elements=None,
            name="prefix",
            op_dict=None,
            producer_op_list=None
        )
    #op.values() for op in graph.get_operations()
    network = {}
    #network['input'] = tf.Variable(np.zeros((1, IMAGE_HEIGHT, IMAGE_WIDTH, COLOR_CHANNELS)), dtype = 'float32')    
    
    for n in graph.get_operations():
        print(n.name)
        network[n.name] = n
    pl = None #temp
    return pl, graph



In [9]:
CONTENT_IMAGE = 'gt/0001_GT.jpg'

from PIL import Image
im = Image.open(CONTENT_IMAGE)
print(im.mode)
width, height = im.size


n,g = load_inception_model("./20180402-114759/20180402-114759.pb", height, width)

RGB
Instructions for updating:
Please file an issue at https://github.com/tensorflow/tensorflow/issues if you depend on this feature.
prefix/embeddings/Maximum/y
prefix/embeddings/Sum/reduction_indices
prefix/InceptionResnetV1/Bottleneck/BatchNorm/moving_variance
prefix/InceptionResnetV1/Bottleneck/BatchNorm/moving_variance/read
prefix/InceptionResnetV1/Bottleneck/BatchNorm/moving_mean
prefix/InceptionResnetV1/Bottleneck/BatchNorm/moving_mean/read
prefix/InceptionResnetV1/Bottleneck/BatchNorm/Const
prefix/InceptionResnetV1/Bottleneck/BatchNorm/beta
prefix/InceptionResnetV1/Bottleneck/BatchNorm/beta/read
prefix/InceptionResnetV1/Bottleneck/BatchNorm/Reshape/shape
prefix/InceptionResnetV1/Bottleneck/weights
prefix/InceptionResnetV1/Bottleneck/weights/read
prefix/InceptionResnetV1/Logits/Flatten/flatten/Reshape/shape/1
prefix/InceptionResnetV1/Logits/Flatten/flatten/strided_slice/stack_2
prefix/InceptionResnetV1/Logits/Flatten/flatten/strided_slice/stack_1
prefix/InceptionResnetV1/Logits/

prefix/InceptionResnetV1/Repeat_1/block17_8/Branch_1/Conv2d_0c_7x1/BatchNorm/cond_1/AssignMovingAvg_1/sub/Switch
prefix/InceptionResnetV1/Repeat_1/block17_8/Branch_1/Conv2d_0c_7x1/BatchNorm/cond_1/AssignMovingAvg/Switch
prefix/InceptionResnetV1/Repeat_1/block17_8/Branch_1/Conv2d_0c_7x1/BatchNorm/cond_1/AssignMovingAvg/sub/Switch
prefix/InceptionResnetV1/Repeat_1/block17_8/Branch_1/Conv2d_0c_7x1/BatchNorm/cond_1/Switch
prefix/InceptionResnetV1/Repeat_1/block17_8/Branch_1/Conv2d_0c_7x1/BatchNorm/cond_1/switch_t
prefix/InceptionResnetV1/Repeat_1/block17_8/Branch_1/Conv2d_0c_7x1/BatchNorm/cond_1/AssignMovingAvg_1/decay
prefix/InceptionResnetV1/Repeat_1/block17_8/Branch_1/Conv2d_0c_7x1/BatchNorm/cond_1/AssignMovingAvg/decay
prefix/InceptionResnetV1/Repeat_1/block17_8/Branch_1/Conv2d_0c_7x1/BatchNorm/cond/pred_id
prefix/InceptionResnetV1/Repeat_1/block17_8/Branch_1/Conv2d_0c_7x1/BatchNorm/cond/FusedBatchNorm_1/Switch_4
prefix/InceptionResnetV1/Repeat_1/block17_8/Branch_1/Conv2d_0c_7x1/BatchN

prefix/InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0a_1x1/BatchNorm/cond/switch_t
prefix/InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0a_1x1/BatchNorm/cond/Const_1
prefix/InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0a_1x1/BatchNorm/cond/Const
prefix/InceptionResnetV1/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/cond_1/pred_id
prefix/InceptionResnetV1/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/cond_1/AssignMovingAvg_1/Switch
prefix/InceptionResnetV1/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/cond_1/AssignMovingAvg_1/sub/Switch
prefix/InceptionResnetV1/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/cond_1/AssignMovingAvg/Switch
prefix/InceptionResnetV1/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/cond_1/AssignMovingAvg/sub/Switch
prefix/InceptionResnetV1/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/cond_1/Switch
prefix/InceptionResnetV1/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/cond_1/switch_t
prefix/InceptionResnetV1/Repeat/blo

prefix/InceptionResnetV1/Repeat_2/block8_1/Branch_0/Conv2d_1x1/BatchNorm/cond/Merge
prefix/InceptionResnetV1/Repeat_2/block8_1/Branch_0/Conv2d_1x1/BatchNorm/cond_1/Switch_1
prefix/InceptionResnetV1/Repeat_2/block8_1/Branch_0/Conv2d_1x1/BatchNorm/cond_1/Identity/Switch
prefix/InceptionResnetV1/Repeat_2/block8_1/Branch_0/Conv2d_1x1/BatchNorm/cond_1/Identity
prefix/InceptionResnetV1/Repeat_2/block8_1/Branch_0/Conv2d_1x1/BatchNorm/cond_1/Merge
prefix/InceptionResnetV1/Repeat_2/block8_1/Branch_0/Conv2d_1x1/Relu
prefix/InceptionResnetV1/Repeat_2/block8_1/concat
prefix/InceptionResnetV1/Repeat_2/block8_1/Conv2d_1x1/Conv2D
prefix/InceptionResnetV1/Repeat_2/block8_1/Conv2d_1x1/BiasAdd
prefix/InceptionResnetV1/Repeat_2/block8_1/mul
prefix/InceptionResnetV1/Repeat_2/block8_1/add
prefix/InceptionResnetV1/Repeat_2/block8_1/Relu
prefix/InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0a_1x1/Conv2D
prefix/InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/FusedBatchNorm_1/Sw

In [10]:
embedded = g.get_tensor_by_name("prefix/embeddings:0")

In [11]:
embedded

<tf.Tensor 'prefix/embeddings:0' shape=(?, 512) dtype=float32>

In [12]:
out = slim.fully_connected(embedded, 3, activation_fn=None, 
                        scope='Out_', reuse=False)

In [142]:
#img all reformat

width_ = 160#int(width/2)
height_ = 160#int(height/2)

for i in range(0,567):
    num = str(i).zfill(4)
    #file = 'gt/'+num+'_GT.jpg'
    file2 = 'nothing/'+num+'_nothing.jpg'
    
    #im = Image.open(file)
    #im = im.resize((width_, height_), Image.ANTIALIAS)
    #im.save("resize_"+file)
    
    im = Image.open(file2)
    im = im.resize((width_, height_), Image.ANTIALIAS)
    im.save("resize_"+file2)


In [97]:
file2 = 'nothing/0567_nothing.jpg'
im = Image.open(file2)
im = im.resize((width_, height_), Image.ANTIALIAS)
im.save("resize_"+file2)

FileNotFoundError: [Errno 2] No such file or directory: 'nothing/0567_nothing.jpg'

In [22]:
def load_images(nums):
    n = len(nums)
    images = np.zeros((n, height_, width_, 3))
    answers = np.zeros((n,3)) 
    it = 0
    for i in nums:
        num = str(i).zfill(4)
        file = 'gt/'+num+'_GT.jpg'
        file2 = 'nothing/'+num+'_nothing.jpg'
    
        img = imageio.imread("resize_"+file2)
        images[it,:,:,:] = img
        answers[it, :] = il[i]
        it = it+1
    #print(images)
    #print(answers)
    return images, answers



In [122]:
img = imageio.imread("resize_nothing/0000_nothing.jpg")

In [123]:
img.shape

(160, 160, 3)

In [124]:
il[44].shape

(3,)

In [ ]:
def load_image(path):
    image = imageio.imread(path)
    # Resize the image for convnet input, there is no change but just
    # add an extra dimension.
    image = np.reshape(image, ((1,) + image.shape))
    # Input to the VGG model expects the mean to be subtracted.
    #image = image - MEAN_VALUES
    return image

def load_data(image_paths, do_random_crop, do_random_flip, image_size, do_prewhiten=True):
    nrof_samples = len(image_paths)
    images = np.zeros((nrof_samples, image_size, image_size, 3))
    for i in range(nrof_samples):
        img = misc.imread(image_paths[i])
        if img.ndim == 2:
            img = to_rgb(img)
        if do_prewhiten:
            img = prewhiten(img)
        img = crop(img, do_random_crop, image_size)
        img = flip(img, do_random_flip)
        images[i,:,:,:] = img
    return images

#inputs = load_data([CONTENT_IMAGE], False, False, height*width)

#input_image = load_image(CONTENT_IMAGE)

width_ = 160#int(width/2)
height_ = 160#int(height/2)

phase_train_placeholder = g.get_tensor_by_name("prefix/phase_train:0")

costlist=[]
import gc
with tf.Session(graph=g) as sess:
#    sess.run(tf.global_variables_initializer())
#    res = sess.run(out, feed_dict=({g.get_tensor_by_name("prefix/input:0"):images, phase_train_placeholder: False}))
   
    def clear_mem():
        try: sess.close()
        except: pass
        sess = tf.InteractiveSession()
        #K.set_session(sess)
        return

    
    ITERATIONS = 1000
    
    images = np.zeros((567, height_, width_, 3))

    Yil = tf.placeholder(tf.float32, [None,3], name= "Yil")
    
    cost = tf.reduce_sum(tf.pow(out - Yil, 2))/(2*70)
    train_step = tf.train.AdamOptimizer().minimize(cost)
    
    for i in range(567):
        num = str(i).zfill(4)
        file = 'gt/'+num+'_GT.jpg'
        file2 = 'nothing/'+num+'_nothing.jpg'
    
        img = imageio.imread("resize_"+file2)
        images[i,:,:,:] = img
   
    for i in range(100):
        
        print('now image id : ', i)
        #il_ = il[i:i+1]

        #images_ = images[i:i+1]

        #print(images_.shape)
        #print(il_.shape)



        sess.run(tf.global_variables_initializer())
        #res = sess.run(out, feed_dict=({g.get_tensor_by_name("prefix/input:0"):images_, phase_train_placeholder: False}))

        print("gogogogoggogo")
        #print(gc.collect())

        samples = 70
        nums = random.sample(range(0,567),samples)

        images, answers = load_images(nums)
        
        images_ = images
        il_ = answers


        
        for it in range(1,ITERATIONS):
            try:

                sess.run(tf.global_variables_initializer())
                sess.run(train_step, feed_dict=({g.get_tensor_by_name("prefix/input:0"):images_, phase_train_placeholder: False, Yil:il_}))
                if it%99 == 0:
                    # Print every 100 iteration.
                    # res = sess.run(out, feed_dict=({g.get_tensor_by_name("prefix/input:0"):images, phase_train_placeholder: False}))

                    print('Iteration %d' % (it))
                    print('cost: ', sess.run(cost, feed_dict=({g.get_tensor_by_name("prefix/input:0"):images_, phase_train_placeholder: False, Yil:il_})))
                    print('out: ', sess.run(out, feed_dict=({g.get_tensor_by_name("prefix/input:0"):images_, phase_train_placeholder: False, Yil:il_}))[0])
                    print('ans: ', il_[0])
            except:
                print('ERROR.... iterate : ', it)
                print('ERROR.... pic id : ', i)
                clear_mem()
                gc.collect()
                
        #sess.run(out)

now image id :  0
gogogogoggogo
Iteration 99
cost:  1007575.3
out:  [ 0.0759976   0.08464093 -0.13800848]
ans:  [381. 688. 398.]
Iteration 198
cost:  1007537.7
out:  [ 0.06765668 -0.06966676  0.11046585]
ans:  [381. 688. 398.]
Iteration 297
cost:  1007517.6
out:  [0.01389744 0.07532392 0.01053949]
ans:  [381. 688. 398.]
Iteration 396
cost:  1007653.94
out:  [-0.03689888 -0.00954796 -0.03848283]
ans:  [381. 688. 398.]
Iteration 495
cost:  1007602.06
out:  [ 0.07394598 -0.0600503   0.06266116]
ans:  [381. 688. 398.]
Iteration 594
cost:  1007675.9
out:  [-0.02966129 -0.07716    -0.01725423]
ans:  [381. 688. 398.]
Iteration 693
cost:  1007676.94
out:  [-0.09863341 -0.02349518 -0.08094074]
ans:  [381. 688. 398.]
Iteration 792
cost:  1007617.1
out:  [ 0.00550081 -0.06194273  0.03057848]
ans:  [381. 688. 398.]
Iteration 891
cost:  1007598.4
out:  [ 0.00224894  0.01139389 -0.05322408]
ans:  [381. 688. 398.]
Iteration 990
cost:  1007640.56
out:  [ 0.01266375 -0.02326773 -0.10322404]
ans:  [381.

Iteration 198
cost:  1091971.8
out:  [-0.0196368  -0.00315012  0.08916283]
ans:  [ 81. 183. 145.]
Iteration 297
cost:  1092068.4
out:  [-0.04521698  0.00319957  0.00090615]
ans:  [ 81. 183. 145.]
Iteration 396
cost:  1091949.0
out:  [ 0.04191201  0.10499465 -0.06504135]
ans:  [ 81. 183. 145.]
Iteration 495
cost:  1091861.0
out:  [-0.00163662  0.14078282  0.04089708]
ans:  [ 81. 183. 145.]
Iteration 594
cost:  1091950.0
out:  [0.03406686 0.03729732 0.09981975]
ans:  [ 81. 183. 145.]
Iteration 693
cost:  1091942.9
out:  [0.05168667 0.11739504 0.01466502]
ans:  [ 81. 183. 145.]
Iteration 792
cost:  1091969.9
out:  [ 0.01983565 -0.04997094  0.08358062]
ans:  [ 81. 183. 145.]
Iteration 891
cost:  1092064.6
out:  [ 0.00034602 -0.0512289   0.01209362]
ans:  [ 81. 183. 145.]
Iteration 990
cost:  1091989.8
out:  [ 0.01001346 -0.00705731  0.11053124]
ans:  [ 81. 183. 145.]
now image id :  9
gogogogoggogo
Iteration 99
cost:  997869.0
out:  [ 0.07892776 -0.00094208  0.08268835]
ans:  [ 559. 1183. 

Iteration 297
cost:  900031.2
out:  [0.06752248 0.07708688 0.0395989 ]
ans:  [180. 378. 271.]
Iteration 396
cost:  900044.25
out:  [-0.10594643 -0.01179172  0.14614178]
ans:  [180. 378. 271.]
Iteration 495
cost:  900062.7
out:  [ 0.06472653 -0.02800607  0.06057124]
ans:  [180. 378. 271.]
Iteration 594
cost:  899993.4
out:  [0.05288465 0.10574831 0.02007596]
ans:  [180. 378. 271.]
Iteration 693
cost:  900039.0
out:  [0.0233556  0.09607556 0.00952176]
ans:  [180. 378. 271.]
Iteration 792
cost:  900065.94
out:  [-0.03622837  0.03638177  0.00187206]
ans:  [180. 378. 271.]
Iteration 891
cost:  900069.1
out:  [0.09333189 0.02730977 0.02689411]
ans:  [180. 378. 271.]
Iteration 990
cost:  899951.75
out:  [0.05782751 0.0764593  0.1284443 ]
ans:  [180. 378. 271.]
now image id :  17
gogogogoggogo
Iteration 99
cost:  1200930.9
out:  [ 0.03166928  0.08751658 -0.12638973]
ans:  [458. 928. 647.]
Iteration 198
cost:  1200887.2
out:  [ 0.03495904  0.03898987 -0.00699088]
ans:  [458. 928. 647.]
Iteratio

Iteration 396
cost:  1109568.9
out:  [0.06365094 0.07889046 0.08368245]
ans:  [448. 554. 305.]
Iteration 495
cost:  1109714.8
out:  [-0.0688049  -0.00785087  0.05935417]
ans:  [448. 554. 305.]
Iteration 594
cost:  1109787.5
out:  [-0.11973526 -0.07615914  0.04056761]
ans:  [448. 554. 305.]
Iteration 693
cost:  1109717.1
out:  [-0.00482676 -0.02269498 -0.04775748]
ans:  [448. 554. 305.]
Iteration 792
cost:  1109662.1
out:  [ 0.06609833  0.03425016 -0.02718518]
ans:  [448. 554. 305.]
Iteration 891
cost:  1109812.8
out:  [-0.10857821 -0.09396747 -0.00175065]
ans:  [448. 554. 305.]
Iteration 990
cost:  1109730.6
out:  [-0.0155338  -0.0651708   0.01412617]
ans:  [448. 554. 305.]
now image id :  25
gogogogoggogo
Iteration 99
cost:  1036256.0
out:  [ 0.07068239  0.02720437 -0.00720109]
ans:  [141. 314. 237.]
Iteration 198
cost:  1036309.1
out:  [-0.02393804  0.06499835 -0.01618022]
ans:  [141. 314. 237.]
Iteration 297
cost:  1036393.6
out:  [-0.08771358 -0.04478091  0.00383779]
ans:  [141. 31